# Generate data with different simulation profiles

<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->

This notebook walks through how to specify the simulations to run in order to generate different data, whether using ones supplied in the repository or sending your own as part of the post to the data generator.

## Prerequisites

Launch this tutorial and all prerequisites using the `all-services` profile of the Docker Compose file for Jupyter-based Druid tutorials. For more information, see the Learn Druid repository [readme](https://github.com/implydata/learn-druid).

## Initialization

Run the next cell to set up the connection to the data generator.

In [37]:
import requests
import json

datagenUrl = "http://datagen:9999"
datagenHeaders = {'Content-Type': 'application/json'}

## Generate data using default configurations

The Data Generator repository contains a number of preconfigured configurations that the community is invited to contribute to.

Use the `/list` endpoint see the available configurations.

In [38]:
display(requests.get(f"{datagenUrl}/list").json())

['clickstream/users_changes.json',
 'clickstream/users_init.json',
 'clickstream/sales_forecast.json',
 'clickstream/clickstream.json',
 'examples/counter.json',
 'examples/variable.json',
 'examples/missing.json',
 'examples/simple.json',
 'examples/object.json',
 'examples/deepthought.json',
 'examples/types.json',
 'examples/list.json',
 'examples/langmap.json',
 'examples/nulls.json',
 'iot/iot_twin_nulls_time_skips.json',
 'iot/iot_twin.json',
 'social/social_posts.json']

Run the following cell to set up a JSON object with a simulation configuration that uses one of the configurations above.

In [39]:
job_name="20240707-iot"

datagen_request = {
    "name": job_name,
    "target": { "type": "file", "path":f"{job_name}.json"},
    "config_file": "iot/iot_twin.json", 
    "time_type": "1969-07-20 20:17:40",
    "time": "10s",
    "concurrency":10
}

Notice that the `config_file` property has been set to the configuration you will use.

Submit the configuration to the data generator to create a file using this configuration by running the following cell.

In [40]:
response = requests.post(f"{datagenUrl}/start", json.dumps(datagen_request), headers=datagenHeaders)

Run the following cell to peek at the generated data.

In [41]:
response = requests.get(f"{datagenUrl}/file/{job_name}.json")
response.raise_for_status()
data = response.content
print(data)

b'{"ts": "1969-07-20 20:17:40.001000", "device": "b8:27:eb:bf:9d:51", "co": "0.004955938648391245", "humidity": "51.0", "light": "false", "lpg": "0.00765082227055719", "motion": "false", "smoke": "0.02041127012241292", "temp": "22.7"}\n{"ts": "1969-07-20 20:17:40.350593", "device": "00:0f:00:70:91:0a", "co": "0.0028400886071015706", "humidity": "76.0", "light": "false", "lpg": "0.005114383400977071", "motion": "false", "smoke": "0.013274836704851536", "temp": "19.700000762939453"}\n{"ts": "1969-07-20 20:17:43.688598", "device": "b8:27:eb:bf:9d:51", "co": "0.004976012340421658", "humidity": "50.9", "light": "false", "lpg": "0.007673227406398091", "motion": "false", "smoke": "0.02047512557617824", "temp": "22.6"}\n{"ts": "1969-07-20 20:17:45.204171", "device": "1c:bf:ce:15:ec:4d", "co": "0.004403026829699689", "humidity": "76.80000305175781", "light": "true", "lpg": "0.007023337145877314", "motion": "false", "smoke": "0.018628225377018803", "temp": "27.0"}\n{"ts": "1969-07-20 20:17:47.37

## Generate data using customer configurations

Instead of using `config_file`, use `config` to send a simulation profile as part of the call to the data generator.

Run the following cell to create a JSON object to send to the simulator.

In [42]:
gen_config = {
  "emitters": [
    {
      "name": "simple_record",
      "dimensions": [
        {
          "type": "string",
          "name": "random_string_column",
          "length_distribution": {
            "type": "constant",
            "value": 13
          },
          "cardinality": 0,
          "chars": "#.abcdefghijklmnopqrstuvwxyz"
        },
        {
          "type": "int",
          "name": "distributed_number",
          "distribution": {
            "type": "uniform",
            "min": 0,
            "max": 1000
          },
          "cardinality": 10,
          "cardinality_distribution": {
            "type": "exponential",
            "mean": 5
          }
        }
      ]
    }
  ],
  "interarrival": {
    "type": "constant",
    "value": 1
  },
  "states": [
    {
      "name": "state_1",
      "emitter": "simple_record",
      "delay": {
        "type": "constant",
        "value": 1
      },
      "transitions": [
        {
          "next": "state_1",
          "probability": 1.0
        }
      ]
    }
  ]
}

The configuration includes:

* The definition of a "simple record" that includes two [dimensions](https://github.com/implydata/druid-datagenerator#dimensions-):
   * A column called "random_string_column" that generates random text.
   * A column called "distributed_number" that contains numbers with a [uniform distribution](https://github.com/implydata/druid-datagenerator#uniform-distribution).
* Definition of the [number of rows per second](https://github.com/implydata/druid-datagenerator#interarrival) within the `interarrival` section.
* A [state](https://github.com/implydata/druid-datagenerator#states-) definition for each of the simulator state machines - in this case, just one.

For more information, see the [documentation](https://github.com/implydata/druid-datagenerator#data-generator-configuration) in the main repository.

In [46]:
job_name="20240707_myownthing"

datagen_request = {
    "name": job_name,
    "target": { "type": "file", "path":f"{job_name}.json"},
    "config": gen_config, 
    "time": "10s",
    "time_type": "SIM",
    "concurrency":10
}

print(datagen_request)

{'name': '20240707_myownthing', 'target': {'type': 'file', 'path': '20240707_myownthing.json'}, 'config': {'emitters': [{'name': 'simple_record', 'dimensions': [{'type': 'string', 'name': 'random_string_column', 'length_distribution': {'type': 'constant', 'value': 13}, 'cardinality': 0, 'chars': '#.abcdefghijklmnopqrstuvwxyz'}, {'type': 'int', 'name': 'distributed_number', 'distribution': {'type': 'uniform', 'min': 0, 'max': 1000}, 'cardinality': 10, 'cardinality_distribution': {'type': 'exponential', 'mean': 5}}]}], 'interarrival': {'type': 'constant', 'value': 1}, 'states': [{'name': 'state_1', 'emitter': 'simple_record', 'delay': {'type': 'constant', 'value': 1}, 'transitions': [{'next': 'state_1', 'probability': 1.0}]}]}, 'time': '10s', 'time_type': 'SIM', 'concurrency': 10}


This example uses the `config` attribute of the request to configure a new custom data generator instead of using a  predefined `config_file`.

In [49]:
response = requests.post(f"{datagenUrl}/start", json.dumps(datagen_request), headers=datagenHeaders)
response.json()

{'message': 'Starting generator for request.',
 'request': {'name': '20240707_myownthing',
  'target': {'type': 'file', 'path': '/files/20240707_myownthing.json'},
  'config': {'emitters': [{'name': 'simple_record',
     'dimensions': [{'type': 'string',
       'name': 'random_string_column',
       'length_distribution': {'type': 'constant', 'value': 13},
       'cardinality': 0,
       'chars': '#.abcdefghijklmnopqrstuvwxyz'},
      {'type': 'int',
       'name': 'distributed_number',
       'distribution': {'type': 'uniform', 'min': 0, 'max': 1000},
       'cardinality': 10,
       'cardinality_distribution': {'type': 'exponential', 'mean': 5}}]}],
   'interarrival': {'type': 'constant', 'value': 1},
   'states': [{'name': 'state_1',
     'emitter': 'simple_record',
     'delay': {'type': 'constant', 'value': 1},
     'transitions': [{'next': 'state_1', 'probability': 1.0}]}],
   'config_file': '__custom__'},
  'time': '10s',
  'time_type': 'SIM',
  'concurrency': 10}}

In [ ]:
Run the cell below to see the data that was generated.

In [53]:
display(requests.get(f"{datagenUrl}/file/{job_name}.json").content[:1024])

b'{"time":"2024-06-07T15:19:58.181","random_string_column":"ojv#pnajyraru","distributed_number":645}\n{"time":"2024-06-07T15:19:59.181","random_string_column":"rxiofhrebpjux","distributed_number":829}\n{"time":"2024-06-07T15:19:59.181","random_string_column":"wzuilinbp.bj.","distributed_number":645}\n{"time":"2024-06-07T15:20:00.181","random_string_column":"pkcxus#leenpp","distributed_number":645}\n{"time":"2024-06-07T15:20:00.181","random_string_column":"iyyvyrvqfgeeu","distributed_number":588}\n{"time":"2024-06-07T15:20:00.181","random_string_column":"riyoeezwxeovb","distributed_number":429}\n{"time":"2024-06-07T15:20:01.181","random_string_column":"ugkbs.swkxask","distributed_number":374}\n{"time":"2024-06-07T15:20:01.181","random_string_column":"pnkagm#mbwcwj","distributed_number":612}\n{"time":"2024-06-07T15:20:01.181","random_string_column":"q#mnakym.xizd","distributed_number":362}\n{"time":"2024-06-07T15:20:01.181","random_string_column":"ke.yatmhscnoz","distributed_number":645}

## Summary

* The `config_file` property is used when running simulations using default configurations.
* Use `config` when sending a custom generator configuration.
* Simulation profiles can include custom columns, states, and different time gaps between rows.

## Learn more

* Read more about the simulator profiles in the [official repository](https://github.com/implydata/druid-datagenerator).